In [28]:
import pandas as pd
import numpy as np

In [29]:
train_final = pd.read_csv('C:/Users/D072202/Desktop/result_data2/simpletriples infer_2019_05_28_12_47_52_414900/final_trainset.csv-strcombined.csv', index_col=['Unnamed: 0'])
test_final = pd.read_csv('C:/Users/D072202/Desktop/result_data2/simpletriples infer_2019_05_28_12_47_52_414900/possible_matches.csv-strcombined.csv', index_col=['Unnamed: 0'])

In [18]:
train_final.head()

,src_0,src_1,tgt_0,tgt_1,label,syntactic_diff,plus_diff,src_tgt_angle,src_angle_to_origin,tgt_angle_to_origin,src_veclen,tgt_veclen,src_tgt_veclen,score
0,-2.957410,1.631149,-6.909093,14.477239,0,0.119403,0.750000,0.682624,-0.270847,0.321995,3.377413,3.377413,13.440157,0.0
1,-2.952701,3.815913,-5.939658,8.986846,0,0.105263,0.808511,0.784008,0.125838,0.197415,4.824898,4.824898,5.971638,2.0
2,-13.303266,15.389165,-2.116501,1.726838,0,0.179245,0.767442,0.777825,0.072381,-0.100530,20.342155,20.342155,17.657941,0.0
3,-18.349653,14.459792,-8.922326,9.369323,0,0.049587,0.827957,0.780309,-0.117195,0.024425,23.362264,23.362264,10.713887,0.0
4,-4.100322,1.781642,-25.052828,17.313156,0,0.052174,0.808989,0.775851,-0.351506,-0.177814,4.470670,4.470670,26.081323,1.0


In [19]:
test_final.head()

,src_0,src_1,tgt_0,tgt_1,label,syntactic_diff,plus_diff,src_tgt_angle,src_angle_to_origin,tgt_angle_to_origin,src_veclen,tgt_veclen,src_tgt_veclen,score
0,-2.957410,1.631149,-6.909093,14.477239,0,0.119403,0.750000,0.682624,-0.270847,0.321995,3.377413,3.377413,13.440157,0.0
1,-2.952701,3.815913,-5.939658,8.986846,0,0.105263,0.808511,0.784008,0.125838,0.197415,4.824898,4.824898,5.971638,2.0
2,-13.303266,15.389165,-2.116501,1.726838,0,0.179245,0.767442,0.777825,0.072381,-0.100530,20.342155,20.342155,17.657941,0.0
3,-18.349653,14.459792,-8.922326,9.369323,0,0.049587,0.827957,0.780309,-0.117195,0.024425,23.362264,23.362264,10.713887,0.0
4,-4.100322,1.781642,-25.052828,17.313156,0,0.052174,0.808989,0.775851,-0.351506,-0.177814,4.470670,4.470670,26.081323,1.0


In [7]:
train_final = train_final.loc[(train_final.plus_diff > 0.8)].append(train_final.loc[train_final.label==1])

In [5]:
len(train.loc[train.label==1])

NameError: name 'train' is not defined

#### Alternative 1: Sample the training data manually.

In [20]:
i = 0
train = None
while i<1:
    i=i+1
    a = train_final.loc[train_final['label']==0].sample(n=5000, replace=True)
    if train is None:
        train = a
    else:
        train = train.append(a, ignore_index=True)

b = train_final.loc[train_final['label']==1].sample(n=614, replace=True)
train = train.append(b, ignore_index=True)

In [19]:
len(train)

100614

#### Alternative 2: Use all available data for training.

In [21]:
train_final = train_final#[['src_tgt_angle','label','src_tgt_veclen','src_angle_to_origin','tgt_angle_to_origin']]
test_final = test_final#[['src_tgt_angle','label','src_tgt_veclen','src_angle_to_origin','tgt_angle_to_origin']]
train = train_final

## Prepare train/test/prediction data

In [22]:
x_train = train.loc[:, train.columns != 'label']
y_train = train['label']

x_test1 = test_final.loc[:, test_final.columns != 'label']
y_test1 = test_final['label']


## Prediction

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from pandas_ml import ConfusionMatrix

In [26]:
model = RandomForestClassifier(n_estimators=10, max_depth=5, random_state=0) #LogisticRegression(solver='lbfgs')
model = model.fit(x_train[['src_tgt_angle','src_tgt_veclen','score']], y_train)
syntactic_model = LogisticRegression(solver='lbfgs')#RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
syntactic_model = syntactic_model.fit(pd.DataFrame(x_train[['syntactic_diff','plus_diff']]), y_train)

In [27]:
prediction = model.predict(x_test1[['src_tgt_angle','src_tgt_veclen','score']])
result = classification_report(np.array(y_test1), prediction, target_names=['false','true'])
print("Results on simple test:")
print(result)
print(ConfusionMatrix( np.array(y_test1), prediction))
print("\n\n--------------------------------------------------------------\n")

prediction = syntactic_model.predict(pd.DataFrame(x_test1[['syntactic_diff','plus_diff']]))
result = classification_report(np.array(y_test1), prediction, target_names=['false','true'])
print("Syntactic matching results on simple test:")
print(result)
print(ConfusionMatrix(np.array(y_test1), prediction))

Results on simple test:
              precision    recall  f1-score   support

       false       0.95      1.00      0.98      3100
        true       0.00      0.00      0.00       154

   micro avg       0.95      0.95      0.95      3254
   macro avg       0.48      0.50      0.49      3254
weighted avg       0.91      0.95      0.93      3254

Predicted     0  1  __all__
Actual                     
0          3100  0     3100
1           154  0      154
__all__    3254  0     3254


--------------------------------------------------------------

Syntactic matching results on simple test:
              precision    recall  f1-score   support

       false       0.95      1.00      0.98      3100
        true       0.00      0.00      0.00       154

   micro avg       0.95      0.95      0.95      3254
   macro avg       0.48      0.50      0.49      3254
weighted avg       0.91      0.95      0.93      3254

Predicted     0  1  __all__
Actual                     
0          3100  

C:\Users\D072202\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\D072202\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
C:\Users\D072202\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\D072202\

In [11]:
test_plus = test.loc[(test.syntactic_diff>0.69) & (test.label==1)]
test_plus = test.append(test.loc[(test.syntactic_diff<0.69) & (test.label==0)], ignore_index=True)
x_test_plus = test_plus.loc[:, test_plus.columns != 'label']
y_test_plus = test_plus['label']
prediction_plus = model.predict(x_test_plus)
result_plus = classification_report(prediction_plus, np.array(y_test_plus), target_names=['false','true'])
print("Results+ on simple test:")
print(result_plus)
print(ConfusionMatrix(prediction_plus, np.array(y_test_plus)))
print("\n\n--------------------------------------------------------------\n")

prediction_plus = syntactic_model.predict(pd.DataFrame(x_test_plus['syntactic_diff']))
result_plus = classification_report(prediction_plus, np.array(y_test_plus), target_names=['false','true'])
print("Syntactic matching results+ on simple test:")
print(result_plus)
print(ConfusionMatrix(prediction_plus, np.array(y_test_plus)))

NameError: name 'test' is not defined

In [16]:
syntactic_model.coef_

array([[-6.75626284]])

In [17]:
syntactic_model.intercept_

array([2.97726496])